## Conexão com o banco de dados

- pip install mysqlclient
- 

### Docs
- SQL Server = https://docs.sqlalchemy.org/en/20/dialects/mssql.html
- MySQL = https://docs.sqlalchemy.org/en/20/dialects/mysql.html


In [36]:
import os
import dotenv
import pandas as pd
from sqlalchemy import create_engine, text

In [37]:
dotenv.load_dotenv('../config/.env')

True

## Criando a conexão

In [38]:
def conectar_db():
    try:
        user = os.environ["user_db"]
        password = os.environ["password_db"]
        host = os.environ["host_db"]

        conexao = f"mysql://{user}:{password}@{host}"
        engine = create_engine(conexao)
        conn = engine.connect()
        print("conexao realizada com sucesso!")

        return conn

    except Exception as e:
        print(f"Erro: {e}")

In [39]:
# validar conexao

conn = conectar_db()

conexao realizada com sucesso!


## Criacao do banco de dados

In [40]:
try:
    database = "RPA"
    query = f"CREATE SCHEMA IF NOT EXISTS {database}"
    conn.execute(text(query))

except Exception as e:
    print(f"Nao foi possivel criar o banco de dados. ERRO: {e}")

In [41]:
# Criando tabela

query_tabela = '''
USE rpa;

CREATE TABLE IF NOT EXISTS TB_PESSOA(
    COD_PESSOA  INT,
    NOME        VARCHAR(200),
    IDADE       INT,
    PROFISSAO   VARCHAR(100)
)
'''
try:
    conn.execute(text(query_tabela))
    print(f"Tabela criada com sucesso no banco {database}")

except Exception as e:
    print(f"Nao foi possivel criar a tabela no banco {database}. ERRO: {e}")

Tabela criada com sucesso no banco RPA


## Coleta de dados de pessoas

In [42]:
df = pd.read_csv("../datasets/dados_pessoas.csv", sep=";")
df.head()

,cod_pessoa,pessoa,idade,profissao
0,1,José,35,Professor
1,2,Ana,52,Enfermeiro
2,3,Maria,45,Advogado
3,4,Pedro,28,Engenheiro
4,5,João,32,Advogado


## Inserir no banco de dados

In [43]:
for registros in df.itertuples():
    cod_pessoa = registros.cod_pessoa
    pessoa = registros.pessoa
    idade = registros.idade
    profissao = registros.profissao

    try:
        query = f""" INSERT INTO TB_PESSOA (COD_PESSOA, NOME, IDADE, PROFISSAO)
                VALUES ({cod_pessoa}, '{pessoa}', {idade}, '{profissao}')
        """

        conn.execute(text(query))
        print(f'Registro inserido com sucesso {pessoa}')


    except Exception as e:
        print(f'nao foi possivel inserir o registro {pessoa}. erro {e}')

Registro inserido com sucesso José
Registro inserido com sucesso Ana
Registro inserido com sucesso Maria
Registro inserido com sucesso Pedro
Registro inserido com sucesso João
Registro inserido com sucesso Carlos
Registro inserido com sucesso Mariana
Registro inserido com sucesso Pedro
Registro inserido com sucesso Marina
Registro inserido com sucesso José
Registro inserido com sucesso Ana
Registro inserido com sucesso Pedro
Registro inserido com sucesso Mariana
Registro inserido com sucesso Maria
Registro inserido com sucesso Carlos
Registro inserido com sucesso Leandro
Registro inserido com sucesso Daniele
Registro inserido com sucesso José
Registro inserido com sucesso Mariana
Registro inserido com sucesso Rodrigo


## Extracao de dado

In [44]:
query =  """
SELECT *
FROM TB_PESSOA;
"""

try:
    dados_coletados = pd.read_sql_query(query, conn)
    print(dados_coletados)

except Exception as e:
    print(f'Nao foi possivel realizar a consulta. erro {e}')

    COD_PESSOA     NOME  IDADE    PROFISSAO
0            1     José     35    Professor
1            2      Ana     52   Enfermeiro
2            3    Maria     45     Advogado
3            4    Pedro     28   Engenheiro
4            5     João     32     Advogado
5            6   Carlos     40       Médico
6            7  Mariana     22   Enfermeiro
7            8    Pedro     50  Programador
8            9   Marina     58       Médico
9           10     José     30    Professor
10          11      Ana     28  Programador
11          12    Pedro     42   Engenheiro
12          13  Mariana     36     Advogado
13          14    Maria     60       Médico
14          15   Carlos     25   Engenheiro
15          16  Leandro     48    Professor
16          17  Daniele     33     Advogado
17          18     José     55   Enfermeiro
18          19  Mariana     20     Advogado
19          20  Rodrigo     38       Médico


In [46]:
query =  """
SELECT *
FROM TB_PESSOA
WHERE PROFISSAO = "Programador";
"""

try:
    dados_coletados = pd.read_sql_query(query, conn)
    print(dados_coletados)

except Exception as e:
    print(f'Nao foi possivel realizar a consulta. erro {e}')

   COD_PESSOA   NOME  IDADE    PROFISSAO
0           8  Pedro     50  Programador
1          11    Ana     28  Programador
